In [ ]:
import json
import random

# Set the number of random entries to select
num_entries = 50000

# Read the original dataset
with open('../json_data/full_coordinate_train.json', 'r') as file:
    data = json.load(file)

len(data)

In [ ]:
data[0]

In [ ]:
# Select random entries
random_entries = random.sample(data, num_entries)

json_elements = []

for entry in random_entries:
    json_object = {
        'id': entry['id'],
        'conversations': entry['conversations']
    }
    json_elements.append(json_object)

# Convert the json_elements list to a JSON string
json_string = json.dumps(json_elements)

# Write the JSON string to a file
with open('small_train.json', 'w', encoding='utf-8') as file:
    json.dump(json_elements, file, ensure_ascii=False, indent=4)

In [19]:
k = 2
with open(f"k_{k}_data.json", "r") as file:
    data = json.load(file)

with open("json_data/small_train.json", "r") as file:
    indexes = json.load(file)

# Updated processing to include both original and augmented data points with images in the prompt
small_data = []

for i, data_index in enumerate(indexes):
    index = int(data_index['id'].split("_")[1])
    for j in range(k):
        quadrant_item = data[index*(k + 1) + j]
        quadrant_item['id'] = f"identity_{i}_{j}"
        small_data.append(quadrant_item)
        
    coordinate_item = data[index*(k + 1) + k]
    coordinate_item['id'] = f"identity_{i}_{k}"
    small_data.append(coordinate_item)

# Save the updated augmented data to a new JSON file
augmented_file_path = f"json_data/small_k_{k}_train.json"
with open(augmented_file_path, "w") as file:
    json.dump(small_data, file, indent=4)
# create augmented data where one data 

In [ ]:
import pandas as pd

# Load the CSV file to examine its structure
csv_file_path = 'data-recursive-main-fixed.csv'
data = pd.read_csv(csv_file_path)
data.head()

In [ ]:
# Adjust the JSON format to include two entries for quadrant and one for position per image sequence
data_k1 = data[data['k'] == 1].copy()
data_k2 = data[data['k'] == 2].copy()
data_k3 = data[data['k'] == 3].copy()

# Merge the data for k=1, 3
merged_data = pd.merge(data_k1, data_k2, on=['img_filename', 'instruction'], suffixes=('_k1', '_k2',))
print(len(merged_data))
merged_data = pd.merge(merged_data, data_k3, on=['img_filename', 'instruction'], suffixes=('', '_k3'))
print(len(merged_data))

In [10]:
import pandas as pd

K = 2

csv_file_path = '../data-recursive-main.csv'
data = pd.read_csv(csv_file_path)
data.head()

datas = []
for k in range(1, K + 2):
    data_k = data[data['k'] == k].copy()
    datas.append(data_k)

merged_data = datas[0]
for k in range(1, K + 1):
    merged_data = pd.merge(merged_data, datas[k], on=['img_filename', 'instruction'], suffixes=(f'_{k-1}', f'_{k}'))

In [ ]:
merged_data.head()

In [ ]:
print(len(merged_data))

In [ ]:
merged_data.head()

In [15]:
import json

In [ ]:


# Reinitialize the JSON data list
json_data = []
for index, row in merged_data.iterrows():
    if row["instruction"] == "":
        continue
    # Entry for quadrant prediction using k=1 image
    entry1 = {
        "id": f"identity_{index}_0",
        "conversations": [
            {
                "from": "user",
                "value": f"Picture 1: <img>data/recursive_augmented_images/{row['new_name_k1']}</img>\n In this UI screenshot, what is the partition of the element corresponding to the command \"{row['instruction']}\" (with quadrant number)?"
            },
            {
                "from": "assistant",
                "value": str(row['quadrant_k1'])
            }
        ]
    }

    # Entry for quadrant prediction using k=2 image
    entry2 = {
        "id": f"identity_{index}_1",
        "conversations": [
            {
                "from": "user",
                "value": f"Picture 1: <img>data/recursive_augmented_images/{row['new_name_k2']}</img>\n In this UI screenshot, what is the partition of the element corresponding to the command \"{row['instruction']}\" (with quadrant number)?"
            },
            {
                "from": "assistant",
                "value": str(row['quadrant_k2'])
            }
        ]
    }

    x = float(row['point'].split(',')[0].split('(')[1])
    y = float(row['point'].split(',')[1].split(')')[0])
    rounded_x = round(x, 2)
    rounded_y = round(y, 2)

    # Entry for position prediction using k=2 image
    entry3 = {
        "id": f"identity_{index}_2",
        "conversations": [
            {
                "from": "user",
                "value": f"Picture 1: <img>data/recursive_augmented_images/{row['new_name']}</img>\n In this UI screenshot, what is the position of the element corresponding to the command \"{row['instruction']}\" (with point)?"
            },
            {
                "from": "assistant",
                "value": f"({rounded_x}, {rounded_y})",
            }
        ]
    }

    json_data.append(entry1)
    json_data.append(entry2)
    json_data.append(entry3)

# Save to a JSON file with the new format
json_output_path = 'k_2_data.json'
with open(json_output_path, 'w') as json_file:
    json.dump(json_data, json_file, indent=4)

json_output_path

In [17]:
K = 1
AUGMENTED_PER_POINT = 3

# Load the CSV file to examine its structure
csv_file_path = '../data-recursive-main.csv'
data = pd.read_csv(csv_file_path)

json_data = []
for i in range(0, len(data), AUGMENTED_PER_POINT):
    for k in range(K):
        row = data.iloc[i + k]
        json_object = {
            "id": f"identity_{i//AUGMENTED_PER_POINT}_{k}",
            "conversations": [
                {
                    "from": "user",
                    "value": f"Picture 1: <img>data/recursive_augmented_images/{row['new_name']}</img>\n In this UI screenshot, what is the partition of the element corresponding to the command \"{row['instruction']}\" (with quadrant number)?"
                },
                {
                    "from": "assistant",
                    "value": str(row['quadrant'])
                }
            ]
        }
        json_data.append(json_object)

    row = data.iloc[i + K]
    
    x = float(row['point'].split(',')[0].split('(')[1])
    y = float(row['point'].split(',')[1].split(')')[0])
    rounded_x = round(x, 2)
    rounded_y = round(y, 2)

    json_object = {
                    "id": f"identity_{i//AUGMENTED_PER_POINT}_{K}",
                    "conversations": [
                        {
                            "from": "user",
                            "value": f"Picture 1: <img>data/recursive_augmented_images/{row['new_name']}</img>\n In this UI screenshot, what is the position of the element corresponding to the command \"{row['instruction']}\" (with point)?"
                        },
                        {
                            "from": "assistant",
                            "value": f"({rounded_x}, {rounded_y})",
                        }
                    ]
                }
    json_data.append(json_object)

with open(f'k_{K}_data.json', 'w') as file:
    json.dump(json_data, file, indent=4)

In [19]:
# make a json list of 9501 random indices from 0 to 113141
import json
import random

# Set the number of random indices to select
num_entries = 9501

# Select random indices
random_entries = random.sample(range(112764), num_entries)

# Convert the random indices to a JSON string
json_string = json.dumps(random_entries)

# Write the JSON string to a file
with open('random_indices.json', 'w', encoding='utf-8') as file:
    json.dump(random_entries, file, ensure_ascii=False, indent=4)

In [ ]:
import json
import random

# Load the JSON file containing the random indices
with open('../json_data/random_indices.json', 'r') as file:
    random_indices = json.load(file)

# make random_indices a set for faster lookup
random_indices = set(random_indices)

test_indices = set()
# make a new json list of 500 different random indices from 0 to 112764
num_entries = 500
for i in range(num_entries):
    print(i)
    ith_entry = random.randint(0, 112764)
    while ith_entry in random_indices or ith_entry in test_indices:
        ith_entry = random.randint(0, 112764)
    test_indices.add(ith_entry)

test_indices = list(test_indices)

# Convert the random indices to a JSON string
json_string = json.dumps(test_indices)

# Write the JSON string to a file
with open('test_indices.json', 'w', encoding='utf-8') as file:
    json.dump(test_indices, file, ensure_ascii=False, indent=4)

In [25]:
import pandas as pd
import json

In [26]:
csv_file_path = 'seeclick_web_bbox.csv'
data = pd.read_csv(csv_file_path)

In [28]:
json_data = []
index = 0
for i in range(0, len(data)):
    row = data.iloc[i]
    if row['instruction'] == ' ' or row['instruction'] == '' or row['instruction'] == '.':
        continue
    bbox = "(" + row['bbox'].split('[')[1].split(']')[0] + ")"
    json_object = {
        "image": "./train_data/"+row['img_filename'],
        "command": row['instruction'],
        "bbox": bbox
    }

    json_data.append(json_object)

with open('seeclick_web_bbox.json', 'w') as file:
    json.dump(json_data, file, indent=4)

In [ ]:
len(json_data)

In [ ]:
import os
import shutil

# Define source and destination directories
source_dir = "../downloads/mobile_images/combined/"
destination_dir = "../images/"

# Ensure the destination directory exists
os.makedirs(destination_dir, exist_ok=True)

# Loop through all files in the source directory
for filename in os.listdir(source_dir):
    # Check if the file is a .jpg image
    if filename.lower().endswith(".jpg"):
        # Construct full file paths
        source_file = os.path.join(source_dir, filename)
        destination_file = os.path.join(destination_dir, filename)
        
        # Move the file
        shutil.move(source_file, destination_file)

print("All .jpg images have been moved to ../images/")

In [ ]:
from PIL import Image
import json
from IPython.display import display
from tqdm import tqdm

# preprocess images for ReVL
data_path = "../json_data/ReVL_text_to_point_10000.json"
with open(data_path, "r") as file:
    data = json.load(file)

for i in tqdm(range(len(data))):
    entry = data[i]
    if entry["type"] == "revl":
        for step in entry["conversations"]:
            if step["from"] == "user":
                zoom = step["zoom"]
                img_path = step["Image"]
                # print(img_path)
                # print(zoom)
                img = Image.open("../../" + img_path)
                width, height = img.size
                # zoom is a normalized bounding box in the format (left, upper, right, lower)
                left = int(zoom[0] * width)
                upper = int(zoom[1] * height)
                right = int(zoom[2] * width)
                lower = int(zoom[3] * height)
                zoomed_img = img.crop((left, upper, right, lower))
                # save the zoomed in image with zoom coordinates in the filename
                new_img_path = img_path.replace(".png", f"_{left}_{upper}_{right}_{lower}.png")
                # show before and after images
                # display(img)
                # display(zoomed_img)
                # print(new_img_path)
                zoomed_img.save("../../" + new_img_path)
                step["value"] = f"<img>{new_img_path}</img>\n" + step["value"]
                # print(step["value"])

# save the updated data to a new JSON file
updated_data_path = data_path.replace(".json", "_with_augmented_images.json")
with open(updated_data_path, "w") as file:
    json.dump(data, file, indent=4)

In [2]:
from PIL import Image
from IPython.display import display

image = Image.open("../images/21942.jpg")

display(image)